<a href="https://colab.research.google.com/github/mgp87/Jupyter_Notebooks_Collection/blob/main/Telugu_gemma_7b_finetuned_sft_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Telugu-LLM-Labs/Telugu-gemma-7b-finetuned-sft

[Telugu-LLM-Labs](https://huggingface.co/Telugu-LLM-Labs)

In this notebook, we will show examples of using `Telugu-gemma-7b-finetuned-sft` a LoRA finetuned model based on [google/gemma-7b](https://huggingface.co/google/gemma-7b) using instruction datasets:
  1. [yahma_alpaca_cleaned_telugu_filtered_and_romanized](https://huggingface.co/datasets/Telugu-LLM-Labs/yahma_alpaca_cleaned_telugu_filtered_and_romanized)
  2. [teknium_GPTeacher_general_instruct_telugu_filtered_and_romanized](https://huggingface.co/datasets/Telugu-LLM-Labs/teknium_GPTeacher_general_instruct_telugu_filtered_and_romanized)

The results shown in this notebook are run on A100. If you plan to run it on T4 please make following changes. (Expect degraded ouput because of quantized version).

1. load_in_4bit = True.
2. `!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"`

Alternatively, you can try `Telugu-gemma-2b-finetuned-sft` a LoRA finetuned model based on [google/gemma-2b](https://huggingface.co/google/gemma-2b).

### Note:

Please make sure you have access to `google/gemma` models on HuggingFace before running the notebook.

## Installation

In [ ]:
!pip install "unsloth[colab-ampere] @git+https://github.com/unslothai/unsloth.git" # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
# !pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git" # Use this for older GPUs (Tesla T4, V100, RTX 20xx)

## HuggingFace Login

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Use unsloth for inference

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "Telugu-LLM-Labs/Telugu-gemma-7b-finetuned-sft",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

FastLanguageModel.for_inference(model)

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:71: UserWarning: Unsloth: Running `ldconfig /usr/lib64-nvidia` to link CUDA.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/692 [00:00<?, ?B/s]

Unsloth: You passed in `google/gemma-7b` and `load_in_4bit = True`.
We shall load `unsloth/gemma-7b-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Gemma patching release 2024.2
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.22.post7. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


...

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/800M [00:00<?, ?B/s]

Unsloth 2024.2 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


## Test Tokenizer

In [ ]:
tokenized_text = tokenizer.tokenize("నేను మార్కెట్ నుండి 100 యాపిల్స్ కొన్నాను.")

print()

```
['నే', 'ను', '▁మ', 'ార్', 'క', 'ె', 'ట్', '▁ను', 'ండి', '▁', '1', '0', '0', '▁య', 'ా', 'ప', 'ి', 'ల్', 'స్', '▁క', 'ొ', 'న్న', 'ా', 'ను', '.']
```

As you can see the tokenizer is pretty exhaustive.

## Input Prompt

In [ ]:
input_prompt = """
### Instruction:
{}

### Input:
{}

### Response:
{}"""

## Reasoning

In [ ]:
instruction = "(5+4)*(2+1)? నాకు దశల వారీ వివరణ ఇవ్వండి." # (5+4)*(2+1)? Give me the step by step explaination.
input = ""

input_text = input_prompt.format(
        instruction, # instruction
        input, # input
        "", # output - leaving blank for generation!
    )

inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 500, use_cache = True)
result = tokenizer.batch_decode(outputs)[0]
print (result)

```
<bos>

### Instruction:
(5+4)*(2+1)? నాకు దశల వారీ వివరణ ఇవ్వండి.

### Input:


### Response:
1. పారెంథసిస్ లోపల వ్యక్తీకరణలను పరిష్కరించండి: (5+4) మరియు (2+1).
2. పరిష్కారాలను గుణించండి: (5+4) * (2+1).
3. ఫలితాన్ని లెక్కించండి: 9 * 3.
4. తుది సమాధానాన్ని పొందడానికి 9ని 3 తో గుణించండి: 27.<eos>
```

In [ ]:
instruction = """నేను మార్కెట్ నుండి 100 యాపిల్స్ కొన్నాను. ఇంటికి వెళ్ళే దారిలో మా అమ్మకి 10 యాపిల్స్, నా స్నేహితుడికి 5 ఆపిల్స్ ఇచ్చాను. ఇప్పుడు నా దగ్గర ఎన్ని ఆపిల్ పండ్లు ఉన్నాయి?"""
# I bought 100 apples from the market. On the way home I gave 10 apples to my mother and 5 apples to my friend. How many apples do I have now?
input = ""

input_text = input_prompt.format(
        instruction, # instruction
        input, # input
        "", # output - leaving blank for generation!
    )

inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 300, use_cache = True)
result = tokenizer.batch_decode(outputs)[0]
print (result)

```
<bos>
### Instruction:
నేను మార్కెట్ నుండి 100 యాపిల్స్ కొన్నాను. ఇంటికి వెళ్ళే దారిలో మా అమ్మకి 10 యాపిల్స్, నా స్నేహితుడికి 5 ఆపిల్స్ ఇచ్చాను. ఇప్పుడు నా దగ్గర ఎన్ని ఆపిల్ పండ్లు ఉన్నాయి?

### Input:


### Response:
మా అమ్మకు ఇచ్చిన ఆపిల్స్ సంఖ్యను మినహాయించి, మార్కెట్లో మిగిలిన ఆపిల్స్ సంఖ్యను (100 - 10 - 5 = 85) లెక్కించండి. అందువల్ల, ఇంటికి వెళ్ళే మార్గంలో మా అమ్మకు ఇచ్చిన ఆపిల్స్ మినహాయించిన తర్వాత మా వద్ద ఇంకా 85 ఆపిల్స్ ఉన్నాయి.<eos>
```

## Content Generation

In [ ]:
instruction = "కాకి మీద చాలా చిన్న కథ రాయండి" # Write a very short children's story about a crow
input = ""

input_text = input_prompt.format(
        instruction, # instruction
        input, # input
        "", # output - leaving blank for generation!
    )

inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 800, use_cache = True)
result = tokenizer.batch_decode(outputs)[0]
print (result)

```
<bos>
### Instruction:
కాకి మీద చాలా చిన్న కథ రాయండి

### Input:


### Response:
ఒకప్పుడు ఒక పల్లెటూరులో అందమైన, పచ్చని పొలాల మధ్య ఒక పెద్ద కాకి ఉండేది. అక్కడి పిల్లలు తరచూ ఆకాశంలోకి ఎగురుతూ, పైకి ఎగురుతూ కనిపించేవారు.

ఒక రోజు, పల్లెటూరులోని ఒక పిల్లవాడు అక్కడికి వచ్చి, కాకిని చూసి ఆశ్చర్యపోయాడు. "కాకి, మీరు అందమైన పక్షి, మీరు ఎక్కడ నుండి వచ్చారు?" అని అడిగాడు.

కాకి కిటికీలోకి దూకింది, తన పక్కనే పడుకుంది మరియు "నాకు ఇక్కడ ఎవరికీ తెలియదు. నాకు ఇక్కడ కూడా ఇష్టం."

కాకికి పిల్లవాడు ఆశ్చర్యపోయాడు మరియు "నాకు ఇక్కడ కూడా ఇష్టం. మీరు అందమైన పక్షి, మీరు ఇక్కడికి వచ్చినందుకు నాకు సంతోషంగా ఉంది."

కాకి కిటికీలోకి దూకింది మరియు పిల్లవాడు వెంటనే పాట వింటాడు. "నాకు ఇక్కడ ఇష్టం. నాకు ఇక్కడికి వచ్చినందుకు ధన్యవాదాలు."

అప్పటి నుండి, పిల్లవాడు తరచూ కాకిని చూడవచ్చు, అది పచ్చని పొలాల పైన ఎగురుతుంది. కానీ అది ఎక్కడి నుంచి వచ్చిందో ఎప్పుడూ అర్థం కాలేదు.<eos>
```

## Question Answering with Context

In [ ]:
instruction = "స్నోఫ్లేక్ సీఈవో ఎవరు?" # who is snowflake ceo?
input = """స్నోఫ్లేక్ (NYSE: SNOW), డేటా క్లౌడ్ కంపెనీ, ఫ్రాంక్ స్లూట్‌మన్ చీఫ్ ఎగ్జిక్యూటివ్ ఆఫీసర్‌గా పదవీ విరమణ చేయాలని నిర్ణయించుకున్నారని మరియు శ్రీధర్ రామస్వామి చీఫ్ ఎగ్జిక్యూటివ్ ఆఫీసర్‌గా మరియు డైరెక్టర్ల బోర్డు సభ్యునిగా వెంటనే అమలులోకి వచ్చినట్లు ఈరోజు ప్రకటించింది. శ్రీధర్ రామస్వామి గతంలో స్నోఫ్లేక్‌లో AI సీనియర్ వైస్ ప్రెసిడెంట్‌గా ఉన్నారు"""

input_text = input_prompt.format(
        instruction, # instruction
        input, # input
        "", # output - leaving blank for generation!
    )

inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 50, use_cache = True)
result = tokenizer.batch_decode(outputs)[0]
print (result)

```
<bos>
### Instruction:
స్నోఫ్లేక్ సీఈవో ఎవరు?

### Input:
స్నోఫ్లేక్ (NYSE: SNOW), డేటా క్లౌడ్ కంపెనీ, ఫ్రాంక్ స్లూట్‌మన్ చీఫ్ ఎగ్జిక్యూటివ్ ఆఫీసర్‌గా పదవీ విరమణ చేయాలని నిర్ణయించుకున్నారని మరియు శ్రీధర్ రామస్వామి చీఫ్ ఎగ్జిక్యూటివ్ ఆఫీసర్‌గా మరియు డైరెక్టర్ల బోర్డు సభ్యునిగా వెంటనే అమలులోకి వచ్చినట్లు ఈరోజు ప్రకటించింది. శ్రీధర్ రామస్వామి గతంలో స్నోఫ్లేక్‌లో AI సీనియర్ వైస్ ప్రెసిడెంట్‌గా ఉన్నారు

### Response:
స్నోఫ్లేక్ సీఈవో శ్రీధర్ రామస్వామి.<eos>
```

## General Question Answering

In [ ]:
instruction = "Share the strategy to grow audience on Twitter in 3 short points?"
input = ""

input_text = input_prompt.format(
        instruction, # instruction
        input, # input
        "", # output - leaving blank for generation!
    )

inputs = tokenizer([input_text], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 800, use_cache = True)
result = tokenizer.batch_decode(outputs)[0]
print (result)

```
<bos>
### Instruction:
Share the strategy to grow audience on Twitter in 3 short points?

### Input:


### Response:
1. **కంటెంట్ సృష్టి మరియు పంపిణీని పెంపొందించండి:** మీరు మీ ప్రేక్షకులకు విలువను అందించే అధిక-నాణ్యత, సంబంధిత మరియు ఆకర్షణీయమైన కంటెంట్ను సృష్టించడం ద్వారా ప్రారంభించండి. మీరు మీ కంటెంట్ను పంచుకునే విధానం గురించి కూడా చాలా అవగాహన పొందాలి. మీరు మీ కంటెంట్ను పంచుకోవడానికి అనుకూలమైన పద్ధతులలో ట్విట్టర్ లైవ్, వీడియోలు, ఇన్ఫోగ్రాఫిక్స్ మరియు మరెన్నో ఉన్నాయి.

2. **ఇతరులతో నిమగ్నం అవ్వండి:** ఇతరులతో నిమగ్నం కావడం అంటే వ్యాఖ్యలు మరియు సందేశాలకు ప్రతిస్పందించడం, ఇతర ట్విట్టర్ వినియోగదారులతో సహకరించడం మరియు మీ పరిధిని పెంచడానికి ఇతర ప్రసిద్ధ ట్విట్టర్ ఖాతాలను పంచుకోవడం. ఇది సంభావ్య అనుచరులను ఆకర్షించడానికి మరియు మీ ప్రస్తుత అనుచరులను నిలుపుకోవటానికి సహాయపడుతుంది.

3. **మీ ప్రేక్షకులను విశ్లేషించండి:** మీ ట్విట్టర్ ఖాతాను పర్యవేక్షించడానికి మరియు మీ పోస్ట్ ల పనితీరును ట్రాక్ చేయడానికి ట్విట్టర్ యొక్క విశ్లేషణ సాధనాలను ఉపయోగించండి. ఇది మీ కంటెంట్ను మెరుగుపరచడానికి మరియు మీ ప్రేక్షకులను మరింత సమర్థవంతంగా లక్ష్యంగా చేసుకోవడానికి మీకు సహాయపడుతుంది. మీ పోస్ట్ ల పనితీరును పర్యవేక్షించడం ద్వారా మరియు అత్యంత పనితీరు కనబరిచిన కంటెంట్ ను పునరుత్పత్తి చేయడం ద్వారా, మీరు మీ ఆన్ లైన్ ఉనికిని పెంచుకోవచ్చు మరియు మీ ప్రేక్షకులను పెంచుకోవచ్చు.<eos>
```